In [1]:
# Dependencies and Setup

import json
import pandas as pd
import requests
import numpy as py
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy import Column, Integer, String, Float, Date
import pymysql
pymysql.install_as_MySQLdb()
from sqlalchemy import create_engine, inspect, func, distinct
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
import sqlite3
from sqlite3 import Error

In [2]:
# ####      Create engine, connection and function to connect to SQLite db

# Create engine and engine connection to SQLite Database

engine = create_engine("sqlite:///game_stats3.sqlite")
conn = engine.connect()

# Create function to try and connect to a sqlite database.  if it doesn't work, create one...

def create_connection(db_file):
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        conn.close()
        
# If connection succeeds

if __name__ == '__main__':
    create_connection("game_stats3.sqlite")

2.6.0


In [3]:
#Bring in data from internet as object then convert ot Json and count number of games

# REQUEST

schedule_response_obj=requests.get("http://data.nba.net/prod/v1/2018/schedule.json")
schedule_response=schedule_response_obj.json()

# Count the number of Games for the season inschedule
games_response_count=schedule_response["league"]["standard"]
seasongame_number=len(games_response_count)
# seasongame_number

In [4]:
###   Collect Elements for URL, append List, create DataFrame, DF_exclude games not played  ###

# Bring in Game Id and GameData ("startDataEastern") elements from NBA Schedule later used to create box_score URL

# Get elements
NBA_Schedule=[]
for item in schedule_response["league"]["standard"]:
    item_dict={}
    item_dict["gameId"]=item["gameId"]
    item_dict["startDateEastern"]=item["startDateEastern"]
    item_dict["statusNum"]=item["statusNum"]

# Append List
    NBA_Schedule.append({"gameid":item_dict["gameId"],
                         "startdateeastern":item_dict["startDateEastern"],
                          "statusnum":item_dict["statusNum"]})


# Convert NBA_Schedule=[] list to DataFrame and modify Datatypes
NBA_Schedule_DF_initial=pd.DataFrame(NBA_Schedule)
NBA_Schedule_DF_initial["gameid"]=NBA_Schedule_DF_initial["gameid"].astype(int)
NBA_Schedule_DF_initial["startdateeastern"]=NBA_Schedule_DF_initial["startdateeastern"].astype(int)
NBA_Schedule_DF_initial["statusnum"]=NBA_Schedule_DF_initial["statusnum"].astype(int)

# Exclude Games not Yet Played
# OPEN: Exclude Pre-season Games
NBA_Schedule_PlayedGM_DF=NBA_Schedule_DF_initial.loc[NBA_Schedule_DF_initial["statusnum"]==3,:]
NBA_Schedule_PlayedGM_DF.head()

,gameid,startdateeastern,statusnum
0,11800001,20180928,3
1,11800002,20180928,3
2,11800003,20180929,3
3,11800004,20180929,3
4,11800005,20180929,3


In [5]:
## Count Games Played, Generate URL for each Game Played, and add to DF: NBA_Schedule_PlayedGM_DF

# count the number of gamesId records collected (note: some are pre-season and some haven't been played)

playedgame_count=len(NBA_Schedule_PlayedGM_DF["gameid"])

In [6]:
# Generate Box_Score_URL and append to List Box_Score_URL_List

Box_score_URL_List=[]

# # Loop to generate Box Score URL using Gameid and GameData ("startDataEastern")

for game_box in range (0,playedgame_count):
    Box_score_URL=F"http://data.nba.net/prod/v1/{NBA_Schedule_PlayedGM_DF['startdateeastern'][game_box]}/00{NBA_Schedule_PlayedGM_DF['gameid'][game_box]}_boxscore.json"

#     Append list
    Box_score_URL_List.append(Box_score_URL)
    
NBA_Schedule_PlayedGM_DF["boxscoreURL"]=Box_score_URL_List
# NBA_Schedule_PlayedGM_DF["boxscoreURL"]=NBA_Schedule_PlayedGM_DF["boxscoreURL"].astype(str)
Box_score_URL
# NBA_Schedule_PlayedGM_DF

C:\Users\Craig\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


'http://data.nba.net/prod/v1/20190131/0021800769_boxscore.json'

In [ ]:
# Extract ONLY player stats from box score
Player_boxscores_list=[]

for i in range (0,playedgame_count):
    box_score_obj=requests.get(NBA_Schedule_PlayedGM_DF["boxscoreURL"][i])   # Derive URL
    box_score_response=box_score_obj.json()                                  # Convert object to response
    for boxscore_playerstats in box_score_response["stats"]["activePlayers"]:    #Loop through players in boxscore
        boxscore_player_dict={}                                                  # Dict to hold one box score loop
        boxscore_player_dict["personId"]=boxscore_playerstats["personId"]
        boxscore_player_dict["teamId"]=boxscore_playerstats["teamId"]
        boxscore_player_dict["points"]=boxscore_playerstats["points"]
        boxscore_player_dict["pos"]=boxscore_playerstats["pos"]
        boxscore_player_dict["min"]=boxscore_playerstats["min"]
        boxscore_player_dict["points"]=boxscore_playerstats["points"]
        boxscore_player_dict["fgm"]=boxscore_playerstats["fgm"]
        boxscore_player_dict["min"]=boxscore_playerstats["min"]
        boxscore_player_dict["points"]=boxscore_playerstats["points"]
        boxscore_player_dict["fgm"]=boxscore_playerstats["fgm"]
        boxscore_player_dict["fga"]=boxscore_playerstats["fga"]
        boxscore_player_dict["fgp"]=boxscore_playerstats["fgp"]
        boxscore_player_dict["ftm"]=boxscore_playerstats["ftm"]
        boxscore_player_dict["fta"]=boxscore_playerstats["fta"]
        boxscore_player_dict["ftp"]=boxscore_playerstats["ftp"]
        boxscore_player_dict["tpm"]=boxscore_playerstats["tpm"]
        boxscore_player_dict["tpa"]=boxscore_playerstats["tpa"]
        boxscore_player_dict["tpp"]=boxscore_playerstats["tpp"]
        boxscore_player_dict["offReb"]=boxscore_playerstats["offReb"]
        boxscore_player_dict["defReb"]=boxscore_playerstats["defReb"]
        boxscore_player_dict["totReb"]=boxscore_playerstats["totReb"]
        boxscore_player_dict["assists"]=boxscore_playerstats["assists"]
        boxscore_player_dict["pFouls"]=boxscore_playerstats["pFouls"]
        boxscore_player_dict["steals"]=boxscore_playerstats["steals"]
        boxscore_player_dict["turnovers"]=boxscore_playerstats["totReb"]
        boxscore_player_dict["blocks"]=boxscore_playerstats["blocks"]
        boxscore_player_dict["plusMinus"]=boxscore_playerstats["plusMinus"]
        boxscore_player_dict["dnp"]=boxscore_playerstats["dnp"]

        Player_boxscores_list.append({"personId":boxscore_player_dict["personId"],
                                    "teamId":boxscore_player_dict["teamId"],
                                    "points":boxscore_player_dict["points"],
                                    "pos":boxscore_player_dict["pos"],
                                    "min":boxscore_player_dict["min"],
                                    "points":boxscore_player_dict["points"],
                                    "fgm":boxscore_playerstats["fgm"],
                                    "min":boxscore_playerstats["min"],
                                    "points":boxscore_playerstats["points"],
                                    "fgm":boxscore_playerstats["fgm"],
                                    "fga":boxscore_playerstats["fga"],
                                    "fgp":boxscore_playerstats["fgp"],
                                    "ftm":boxscore_playerstats["ftm"],
                                    "fta":boxscore_playerstats["fta"],
                                    "ftp":boxscore_playerstats["ftp"],
                                    "tpm":boxscore_playerstats["tpm"],
                                    "tpa":boxscore_playerstats["tpa"],
                                    "tpp":boxscore_playerstats["tpp"],
                                    "offReb":boxscore_playerstats["offReb"],
                                    "defReb":boxscore_playerstats["defReb"],
                                    "totReb":boxscore_playerstats["totReb"],
                                    "assists":boxscore_playerstats["assists"],
                                    "pFouls":boxscore_playerstats["pFouls"],
                                    "steals":boxscore_playerstats["steals"],
                                    "turnovers":boxscore_playerstats["totReb"],
                                    "blocks":boxscore_playerstats["blocks"],
                                    "plusMinus":boxscore_playerstats["plusMinus"],
                                    "dnp":boxscore_playerstats["dnp"]})
        
Player_boxscores_df=pd.DataFrame(Player_boxscores_list)
Player_boxscores_df

In [ ]:
#Load NBA_TEAMS table
Player_boxscores_df.to_sql('playerstats_boxscore',conn, if_exists='replace', index=False)

In [ ]:
mnghjh

In [ ]:
  boxscore_player_dict["personid"]=box_score_response["stats"]["activePlayers"]["personId"]
    boxscore_player_dict["teamid"]=box_score_response["stats"]["activePlayers"]["teamId"]
    boxscore_player_dict["points"]=box_score_response["stats"]["activePlayers"]["points"]
    boxscore_player_dict["pos"]=box_score_response["stats"]["activePlayers"]["pos"]
    boxscore_player_dict["min"]=box_score_response["stats"]["activePlayers"]["min"]
    boxscore_player_dict["points"]=box_score_response["basicGameData"]["points"]
    boxscore_player_dict["fgm"]=box_score_response["basicGameData"]["fgm"]
    boxscore_player_dict["min"]=box_score_response["basicGameData"]["min"]
    boxscore_player_dict["points"]=box_score_response["basicGameData"]["points"]
    boxscore_player_dict["fgm"]=box_score_response["basicGameData"]["fgm"]
    boxscore_player_dict["fga"]=box_score_response["basicGameData"]["fga"]
    boxscore_player_dict["fgp"]=box_score_response["basicGameData"]["fgp"]
    boxscore_player_dict["ftm"]=box_score_response["basicGameData"]["ftm"]
    boxscore_player_dict["fta"]=box_score_response["basicGameData"]["fta"]
    boxscore_player_dict["ftp"]=box_score_response["basicGameData"]["ftp"]
    boxscore_player_dict["tpm"]=box_score_response["basicGameData"]["tpm"]
    boxscore_player_dict["tpa"]=box_score_response["basicGameData"]["tpa"]
    boxscore_player_dict["tpp"]=box_score_response["basicGameData"]["tpp"]
    boxscore_player_dict["offReb"]=box_score_response["basicGameData"]["offReb"]
    boxscore_player_dict["defReb"]=box_score_response["basicGameData"]["defReb"]
    boxscore_player_dict["totReb"]=box_score_response["basicGameData"]["totReb"]
    boxscore_player_dict["assists"]=box_score_response["basicGameData"]["assists"]
    boxscore_player_dict["pFouls"]=box_score_response["basicGameData"]["pFouls"]
    boxscore_player_dict["steals"]=box_score_response["basicGameData"]["steals"]
    boxscore_player_dict["turnovers"]=box_score_response["basicGameData"]["totReb"]
    boxscore_player_dict["blocks"]=box_score_response["basicGameData"]["blocks"]
    boxscore_player_dict["plusMinus"]=box_score_response["plusMinus"]
    boxscore_player_dict["dnp"]=box_score_response["dnp"]

    Player_boxscores_list.append({"personid":boxscore_player_dict["personid"],
                                "teamid":boxscore_player_dict["teamid"],
                                "points":boxscore_player_dict["points"],
                                "pos":boxscore_player_dict["pos"],
                                "min":boxscore_player_dict["min"],
                                "points":boxscore_player_dict["points"],
                                "fgm":boxscore_player_dict["fgm"],
                                "min":boxscore_player_dict["min"],
                                "points":boxscore_player_dict["points"],
                                "fgm":boxscore_player_dict["fgm"],
                                "fga":boxscore_player_dict["fga"],
                                "fgp":boxscore_player_dict["fgp"],
                                "ftm":boxscore_player_dict["ftm"],
                                "fta":boxscore_player_dict["fta"],
                                "ftp":boxscore_player_dict["ftp"],
                                "tpm":boxscore_player_dict["tpm"],
                                "tpa":boxscore_player_dict["tpa"],
                                "tpp":boxscore_player_dict["tpp"],
                                "offreb":boxscore_player_dict["offReb"],
                                "defreb":boxscore_player_dict["defReb"],
                                "totreb":boxscore_player_dict["totReb"],
                                "assists":boxscore_player_dict["assists"],
                                "pfouls":boxscore_player_dict["pFouls"],
                                "steals":boxscore_player_dict["steals"],
                                "turnovers":boxscore_player_dict["totReb"],
                                "blocks":boxscore_player_dict["blocks"],
                                "plusminus":boxscore_player_dict["plusMinus"],
                                "dnp":boxscore_player_dict["dnp"]})

In [ ]:
Player_boxscores_df=pd.DataFrame(Player_boxscores_list)
Player_boxscores_df["gameid"]=Player_boxscores_df["gameid"].astype(int)
Player_boxscores_df["teamid"]=Player_boxscores_df["teamid"].astype(int)
Player_boxscores_df["fastbreakpoints"]=Player_boxscores_df["fastbreakpoints"].astype(float)
Player_boxscores_df["pointsinpaint"]=Player_boxscores_df["pointsinpaint"].astype(float)
Player_boxscores_df["secondchancepoints"]=Player_boxscores_df["secondchancepoints"].astype(float)
Player_boxscores_df["pointsoffturnovers"]=Player_boxscores_df["pointsoffturnovers"].astype(float)
Player_boxscores_df["points"]=Player_boxscores_df["points"].astype(float)
Player_boxscores_df["fgm"]=Player_boxscores_df["fgm"].astype(float)
Player_boxscores_df["fga"]=Player_boxscores_df["fga"].astype(float)
Player_boxscores_df["fgp"]=Player_boxscores_df["fgp"].astype(float)
Player_boxscores_df["ftm"]=Player_boxscores_df["ftm"].astype(float)
Player_boxscores_df["fta"]=Player_boxscores_df["fta"].astype(float)
Player_boxscores_df["ftp"]=Player_boxscores_df["ftp"].astype(float)
Player_boxscores_df["tpm"]=Player_boxscores_df["tpm"].astype(float)
Player_boxscores_df["tpa"]=Player_boxscores_df["tpa"].astype(float)
Player_boxscores_df["tpp"]=Player_boxscores_df["tpp"].astype(float)
Player_boxscores_df["offreb"]=Player_boxscores_df["offreb"].astype(float)
Player_boxscores_df["defreb"]=Player_boxscores_df["defreb"].astype(float)
Player_boxscores_df["totreb"]=Player_boxscores_df["totreb"].astype(float)
Player_boxscores_df["assists"]=Player_boxscores_df["assists"].astype(float)
Player_boxscores_df["pfouls"]=Player_boxscores_df["pfouls"].astype(float)
Player_boxscores_df["steals"]=Player_boxscores_df["steals"].astype(float)
Player_boxscores_df["turnovers"]=Player_boxscores_df["turnovers"].astype(float)
Player_boxscores_df["blocks"]=Player_boxscores_df["blocks"].astype(float)
Player_boxscores_df["plusminus"]=Player_boxscores_df["plusminus"].astype(float)
Player_boxscores_df.dtypes        

In [ ]:
#Load NBA_TEAMS table
Team_boxscores_df.to_sql('playerstats_boxscore',conn, if_exists='replace', index=False)